# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [2]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [3]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
        
    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word[::2].replace("_",".")
        
        # find length of passed word
        len_word = len(clean_word)
        
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        
        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue
                
            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word,dict_word):
                new_dictionary.append(dict_word)
        
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        
        dict_unique_letters = []
        for dict_word in new_dictionary:
            dict_unique_letters += list(set(dict_word))

        # count occurrence of all characters in possible word matches
        full_dict_string = "".join(dict_unique_letters)
        
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common()                   
        
        guess_letter = '!'
        
        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        for letter,instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                if(guess_letter == '!'):
                    guess_letter = letter
                else:
                    if(abs(instance_count/len(new_dictionary) - 0.5) < abs(c[guess_letter]/len(new_dictionary) - 0.5)):
                        guess_letter = letter
            
        # if no word matches in training dictionary, default back to ordering of full dictionary
        if guess_letter == '!':
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break            
        
        return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [4]:
api = HangmanAPI(access_token="84c90972d1c50dc5c81ffc54d53004", timeout=2000)


KeyboardInterrupt: 

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [ ]:
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


Successfully start a new game! Game ID: 58c076545b57. # of tries remaining: 6. Word: _ _ _ _ _ _ _ .
Guessing letter: a
Sever response: {'game_id': '58c076545b57', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ a _ _ _ '}
Guessing letter: e
Sever response: {'game_id': '58c076545b57', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ a _ _ _ '}
Guessing letter: r
Sever response: {'game_id': '58c076545b57', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ r a _ _ _ '}
Guessing letter: s
Sever response: {'game_id': '58c076545b57', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ r a _ _ _ '}
Guessing letter: i
Sever response: {'game_id': '58c076545b57', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ r a _ _ i '}
Guessing letter: n
Sever response: {'game_id': '58c076545b57', 'status': 'ongoing', 'tries_remains': 4, 'word': 'n _ r a _ _ i '}
Guessing letter: o
Sever response: {'game_id': '58c076545b57', 'status': 'ongoing', 'tries_remains': 3, 'word': 'n _ r 

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    #api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

: 

: 

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [ ]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

NameError: name 'api' is not defined

In [ ]:
import random
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Load words from words.txt into a list
with open('words_250000_train.txt', 'r') as f:
    words_list = f.read().splitlines()

# Hangman class
class Hangman:
    def __init__(self, word):
        print("New word: ", word, "\n")
        self.word = word
        self.guesses = []
        self.max_attempts = 6
        self.attempts_left = self.max_attempts

    def guess(self, letter):
        self.guesses.append(letter)
        if letter not in self.word:
            self.attempts_left -= 1

    def is_word_guessed(self):
        return all(letter in self.guesses for letter in self.word)

    def get_current_state(self):
        return ''.join([letter if letter in self.guesses else '_' for letter in self.word])

    def get_remaining_attempts(self):
        return self.attempts_left

    def is_game_over(self):
        return self.attempts_left == 0 or self.is_word_guessed()

# DQN Agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = []
        self.gamma = 0.9  # discount factor
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state, available_actions):
        if np.random.rand() <= self.epsilon:
            return random.choice(available_actions)
        q_values = self.model.predict(state)
        available_q_values = q_values[0][available_actions]
        return available_actions[np.argmax(available_q_values)]

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state.reshape(1, -1))[0]))
            target_f = self.model.predict(state.reshape(1, -1))
            target_f[0][action] = target
            self.model.fit(state.reshape(1, -1), target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# RL Training
def train_rl_agent(words_list, epochs=250000, batch_size=32):
    # Create a dictionary for indexing characters for actions
    chars = sorted(list(set(''.join(words_list))))
    char_to_idx = {char: idx for idx, char in enumerate(chars)}
    idx_to_char = {idx: char for char, idx in char_to_idx.items()}

    state_size = len(chars)
    action_size = len(chars)

    agent = DQNAgent(state_size, action_size)
    random.shuffle(words_list)
    for epoch in range(epochs):
        word=words_list[epoch]
        hangman = Hangman(word)
        done = False

        while not done:
            state = np.array([1 if idx_to_char[idx] in hangman.guesses else 0 for idx in range(state_size)])
            available_actions = [idx for idx in range(len(chars)) if idx_to_char[idx] not in hangman.guesses]
            action = agent.act(state.reshape(1, -1), available_actions)

            letter = idx_to_char[action]
            hangman.guess(letter)

            next_state = np.array([1 if idx_to_char[idx] in hangman.guesses else 0 for idx in range(state_size)])
            reward = -1 if letter not in word else 0
            done = hangman.is_game_over()

            agent.remember(state, action, reward, next_state, done)

        if epoch > batch_size:
            agent.replay(batch_size)

        print(f"Epoch: {epoch + 1}/{epochs}, Word: {word}, Epsilon: {agent.epsilon}")

    return agent

if __name__ == "__main__":
    # Training the RL agent
    trained_agent = train_rl_agent(words_list)

    # Save the trained agent's model
    trained_agent.model.save('hangman_dqn_model.h5')


: 

In [ ]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('hangman_dqn_model.h5')

# Dictionary for indexing characters for actions
chars = sorted(list(set('abcdefghijklmnopqrstuvwxyz')))
char_to_idx = {char: idx for idx, char in enumerate(chars)}
idx_to_char = {idx: char for char, idx in char_to_idx.items()}

# Hangman class
class Hangman:
    def __init__(self, word):
        self.word = word
        self.guesses = []
        self.max_attempts = 6
        self.attempts_left = self.max_attempts

    def guess(self, letter):
        self.guesses.append(letter)
        if letter not in self.word:
            self.attempts_left -= 1

    def is_word_guessed(self):
        return all(letter in self.guesses for letter in self.word)

    def get_current_state(self):
        return ''.join([letter if letter in self.guesses else '_' for letter in self.word])

    def get_remaining_attempts(self):
        return self.attempts_left

    def is_game_over(self):
        return self.attempts_left == 0 or self.is_word_guessed()

# Function to play Hangman using the trained DQN model
def play_hangman():
    word = random.choice(words_list)
    hangman = Hangman(word)
    state_size = len(chars)
    state = np.zeros(state_size)

    while not hangman.is_game_over():
        # Exclude previously guessed letters from the action space
        available_actions = [idx for idx in range(len(chars)) if idx_to_char[idx] not in hangman.guesses]

        # If all letters have been guessed, break the loop
        if not available_actions:
            break

        # Get the action (letter) from the DQN model
        action = model.predict(state.reshape(1, -1))
        action_idx = np.argmax(action[0][available_actions])
        letter = idx_to_char[available_actions[action_idx]]

        print(f"Guessing: {letter}")
        hangman.guess(letter)
        state = np.array([1 if idx_to_char[idx] in hangman.guesses else 0 for idx in range(state_size)])
        print(f"Current State: {hangman.get_current_state()}")
        print(f"Attempts Left: {hangman.get_remaining_attempts()}")

    if hangman.is_word_guessed():
        print(f"Congratulations! You guessed the word: {word}")
    else:
        print(f"Game Over! The word was: {word}")

if __name__ == "__main__":
    play_hangman()


In [9]:
def new_word():
    with open('words_250000_train.txt', 'r') as f:
        words = f.readlines()
        return random.choice(words).strip()

In [ ]:
#frequency based guesser

In [15]:
def count_single(wordlist):
    count = {}
    for word in wordlist:
        for letter in word:
            if letter != '\n':
                if letter in count:
                    count[letter] += 1
                else:
                    count[letter] = 1
    return count

In [35]:
def count_1x1(wordlist):
    dict = {}
    for i in range(26):
        dict[chr(i+97)] = {}
        for word in wordlist:
            if chr(i+97) in word:
                for j in word:
                    if j != chr(i+97) and j != '\n':
                        if j in dict[chr(i+97)]:
                            dict[chr(i+97)][j] += 1
                        else:
                            dict[chr(i+97)][j] = 1
    return dict

In [86]:
def count_2x1(wordlist):
    dict={}
    for word in wordlist:
        for i in range(len(word)-1):
            phrase=word[i:i+2]
            if(phrase[0]!='\n' and phrase[1]!='\n'):
                if phrase not in dict:
                    dict[phrase]={}
                for j in word:
                    if j not in phrase and j!='\n':
                        if j in dict[phrase]:
                            dict[phrase][j]+=1
                        else:
                            dict[phrase][j]=1
    return dict


In [97]:
def count_3x1(wordlist):
    dict={}
    for word in wordlist:
        for i in range(len(word)-2):
            phrase=word[i:i+3]
            if(phrase[0]!='\n' and phrase[1]!='\n' and phrase[2]!='\n'):
                if phrase not in dict:
                    dict[phrase]={}
                for j in word:
                    if j not in phrase and j!='\n':
                        if j in dict[phrase] :
                            dict[phrase][j]+=1
                        else:
                            dict[phrase][j]=1
    return dict

In [26]:
def count_nx1(wordlist):
    dict={}
    for word in wordlist:
        l=len(word)
        if(l not in dict):
            dict[l]={}
        for i in word:
            if(i != '\n'):
                if(i in dict[l]):
                    dict[l][i]+=1
                else:
                    dict[l][i]=1
    return dict

In [12]:
with open('words_250000_train.txt', 'r') as f:
    wordlist = f.readlines()

In [16]:
c1=count_single(wordlist)

In [51]:
cn=count_nx1(wordlist)

In [28]:
for i in cn.keys():
    l=cn[i]
    ls=sorted(l.items(), key=lambda x: x[1], reverse=True)
    cn[i]=ls

In [30]:
sorted(cn.items(), key=lambda x: x[0])

[(2,
  [('c', 1),
   ('d', 1),
   ('e', 1),
   ('f', 1),
   ('g', 1),
   ('h', 1),
   ('i', 1),
   ('j', 1),
   ('k', 1),
   ('m', 1),
   ('o', 1),
   ('p', 1),
   ('r', 1),
   ('s', 1),
   ('v', 1),
   ('w', 1),
   ('y', 1)]),
 (3,
  [('c', 26),
   ('w', 26),
   ('d', 25),
   ('r', 25),
   ('h', 25),
   ('e', 24),
   ('i', 24),
   ('a', 22),
   ('o', 22),
   ('u', 22),
   ('l', 21),
   ('n', 21),
   ('f', 21),
   ('s', 20),
   ('x', 20),
   ('t', 20),
   ('p', 19),
   ('v', 19),
   ('b', 17),
   ('k', 17),
   ('m', 17),
   ('q', 16),
   ('g', 15),
   ('y', 15),
   ('j', 15),
   ('z', 14)]),
 (4,
  [('a', 533),
   ('s', 448),
   ('c', 397),
   ('e', 383),
   ('o', 364),
   ('i', 351),
   ('t', 349),
   ('m', 347),
   ('d', 337),
   ('p', 333),
   ('r', 318),
   ('l', 287),
   ('b', 268),
   ('n', 247),
   ('u', 244),
   ('f', 223),
   ('g', 220),
   ('h', 187),
   ('w', 146),
   ('v', 146),
   ('y', 132),
   ('k', 118),
   ('x', 73),
   ('j', 66),
   ('q', 46),
   ('z', 40)]),
 (5,
  [

In [36]:
c2=count_1x1(wordlist)

In [37]:
c2

{'a': {'s': 81054,
  'c': 57148,
  'h': 35649,
  'e': 124155,
  'n': 92933,
  'g': 30394,
  'd': 42684,
  'l': 79002,
  'u': 39742,
  'i': 106863,
  't': 87351,
  'm': 39677,
  'o': 77679,
  'p': 39305,
  'r': 90543,
  'v': 11945,
  'k': 10080,
  'w': 8736,
  'f': 13335,
  'b': 24669,
  'x': 3520,
  'y': 24142,
  'j': 2190,
  'z': 5549,
  'q': 2626},
 'b': {'a': 31399,
  'r': 24747,
  'o': 22675,
  'n': 21545,
  's': 21823,
  'u': 15334,
  'g': 6760,
  'd': 11359,
  'e': 38398,
  'h': 7164,
  'c': 11104,
  'x': 699,
  'i': 28262,
  't': 18604,
  'k': 3286,
  'l': 23322,
  'y': 6142,
  'f': 2857,
  'm': 8384,
  'p': 5361,
  'j': 588,
  'v': 2257,
  'z': 1274,
  'q': 431,
  'w': 2387},
 'c': {'a': 66710,
  'h': 23442,
  'e': 77179,
  'n': 52174,
  'r': 51013,
  'o': 60359,
  'i': 70729,
  'b': 10376,
  's': 46619,
  'x': 1752,
  't': 48859,
  'u': 26583,
  'k': 6507,
  'l': 42485,
  'y': 15333,
  'm': 20023,
  'd': 21127,
  'v': 5915,
  'p': 22858,
  'g': 11700,
  'j': 1009,
  'f': 7182,

In [40]:
c1

{'a': 179837,
 's': 148462,
 'c': 89367,
 'h': 58051,
 'e': 233745,
 'n': 152259,
 'g': 51850,
 'd': 74856,
 'l': 122431,
 'u': 77304,
 'i': 184746,
 't': 137277,
 'm': 62191,
 'o': 150052,
 'p': 65785,
 'r': 149228,
 'v': 21057,
 'k': 18685,
 'w': 17732,
 'f': 26431,
 'b': 39840,
 'x': 6050,
 'y': 40985,
 'j': 3790,
 'z': 8749,
 'q': 3986}

In [41]:
c1=sorted(c1.items(), key=lambda x: x[1], reverse=True)

In [42]:
c1

[('e', 233745),
 ('i', 184746),
 ('a', 179837),
 ('n', 152259),
 ('o', 150052),
 ('r', 149228),
 ('s', 148462),
 ('t', 137277),
 ('l', 122431),
 ('c', 89367),
 ('u', 77304),
 ('d', 74856),
 ('p', 65785),
 ('m', 62191),
 ('h', 58051),
 ('g', 51850),
 ('y', 40985),
 ('b', 39840),
 ('f', 26431),
 ('v', 21057),
 ('k', 18685),
 ('w', 17732),
 ('z', 8749),
 ('x', 6050),
 ('q', 3986),
 ('j', 3790)]

In [43]:
s1=""
for i in c1:
    s1+=i[0]

In [44]:
s1

'eianorstlcudpmhgybfvkwzxqj'

In [45]:
len(s1)

26

In [63]:
with open ('count.txt','w') as f:
    f.write(s1)
    f.write('\n')

In [52]:
cn=sorted(cn.items(), key=lambda x: x[0])

In [54]:
cn=[(1,{'a':1,'i':1})]+cn

In [58]:
cn

[(1, [('a', 1), ('i', 1)]),
 (2,
  [('c', 1),
   ('d', 1),
   ('e', 1),
   ('f', 1),
   ('g', 1),
   ('h', 1),
   ('i', 1),
   ('j', 1),
   ('k', 1),
   ('m', 1),
   ('o', 1),
   ('p', 1),
   ('r', 1),
   ('s', 1),
   ('v', 1),
   ('w', 1),
   ('y', 1)]),
 (3,
  [('c', 26),
   ('w', 26),
   ('d', 25),
   ('r', 25),
   ('h', 25),
   ('e', 24),
   ('i', 24),
   ('a', 22),
   ('o', 22),
   ('u', 22),
   ('l', 21),
   ('n', 21),
   ('f', 21),
   ('s', 20),
   ('x', 20),
   ('t', 20),
   ('p', 19),
   ('v', 19),
   ('b', 17),
   ('k', 17),
   ('m', 17),
   ('q', 16),
   ('g', 15),
   ('y', 15),
   ('j', 15),
   ('z', 14)]),
 (4,
  [('a', 533),
   ('s', 448),
   ('c', 397),
   ('e', 383),
   ('o', 364),
   ('i', 351),
   ('t', 349),
   ('m', 347),
   ('d', 337),
   ('p', 333),
   ('r', 318),
   ('l', 287),
   ('b', 268),
   ('n', 247),
   ('u', 244),
   ('f', 223),
   ('g', 220),
   ('h', 187),
   ('w', 146),
   ('v', 146),
   ('y', 132),
   ('k', 118),
   ('x', 73),
   ('j', 66),
   ('q', 4

In [57]:
for i in range(len(cn)):
    l=cn[i][1]
    ls=sorted(l.items(), key=lambda x: x[1], reverse=True)
    cn[i]=(cn[i][0],ls)

In [59]:
cn_dict={}
for i in cn:
    l=i[1]
    s=""
    for j in l:
        s+=j[0]
    cn_dict[i[0]]=s

In [60]:
cn_dict

{1: 'ai',
 2: 'cdefghijkmoprsvwy',
 3: 'cwdrheiaoulnfsxtpvbkmqgyjz',
 4: 'asceoitmdprlbnufghwvykxjqz',
 5: 'aeosirltnudmcpbhgykfwvzjxq',
 6: 'aesorilntudcymhbpgkfwvzjxq',
 7: 'eairsonltudcmhpgbykfwvzjxq',
 8: 'eairsnoltducmhpgbykfwvzjxq',
 9: 'eaisrnotldcumphgbyfkwvzxjq',
 10: 'eiasrontlcdumphgbyfkwvzxqj',
 11: 'eiarnostlcdupmhgbyfvwkzxqj',
 12: 'eianrotslcdupmhgybfvwkzxqj',
 13: 'eianortslcudpmhgybfvwkzxqj',
 14: 'einaotrslcupdmhgybfvwzkxqj',
 15: 'einaotsrlcupdmhgybfvzwqxkj',
 16: 'ienaotsrlcupmhdygbvfzqwxkj',
 17: 'einosatrlcpuhmydgbvfzqxwkj',
 18: 'eionastrlcpuhmydgbvfzqxwkj',
 19: 'ieonstarlcpuhmydgbvfzqxwkj',
 20: 'ieoantsrlcphumydgbvfzwxqkj',
 21: 'eoiatnrlscphmyudgvbfzqwjxk',
 22: 'oietasnrlchypmudgbvfzqxjkw',
 23: 'oierntlsachpydmugvbxfzjw',
 24: 'ioeltacrndysphmufgvxbzq',
 25: 'oiertlachpnysdmgufbxz',
 26: 'eanislrthcpobmdgu',
 28: 'orectysihdxmplna',
 29: 'alhfembrcingy',
 30: 'tienrylmhcoap'}

In [64]:
with open ('count.txt','a') as f:
    f.write("----------------------------------\n")
    for i in cn_dict.keys():
        f.write(cn_dict[i])
        f.write('\n')

In [65]:
c1x1=count_1x1(wordlist)

In [67]:
for i in c1x1.keys():
    l=c1x1[i]
    ls=sorted(l.items(), key=lambda x: x[1], reverse=True)
    c1x1[i]=ls

In [69]:
for i in c1x1.keys():
    l=c1x1[i]
    s=""
    for j in l:
        s+=j[0]
    c1x1[i]=s

In [70]:
c1x1

{'a': 'einrtslocdumphgbyfvkwzxqj',
 'b': 'eairlosntudcmhgypkfwvzxjq',
 'c': 'eiaonrtsluhpdmygbfkvwzxqj',
 'd': 'eianorsltucpmhgbyfwvkzxqj',
 'e': 'irasntolcdupmhgbyfvwkzxqj',
 'f': 'eironalstudcgmhpybwvkzxqj',
 'g': 'ienaorlstudchpmybfwvkzxjq',
 'h': 'eaiosrtnclpdmuygbwfkvzxjq',
 'i': 'enastrolcdpumghybfvkwzxqj',
 'j': 'eainoustrlcdgbmypkhvwfzqx',
 'k': 'eaisnrolctduhbmpgwyfvjzqx',
 'l': 'eiaonstrcudpmyghbfvwkzxqj',
 'm': 'eiaonsrtlcudphygbfkvwzxjq',
 'n': 'eiaostrlcudgmphybfvkwzxqj',
 'o': 'einrastlcpdumhgybfvwkzxjq',
 'p': 'eioarstnlchudmygbfvkwxzqj',
 'q': 'uieasntrlocdympgbhfvkzwxj',
 'r': 'eaiotsnlcudpmhgbyfvwkzxqj',
 's': 'eiaonrtlucpdmhgbyfvkwzxqj',
 't': 'eiaonrslcuphmdgybfvwkzxqj',
 'u': 'ensiartolcdpmgbhyfvkqwzxj',
 'v': 'eiranosltcudgpymbhfwkzxqj',
 'w': 'eorainlstdhgcubmpkyfvzjxq',
 'x': 'eiaotnrslcpudymhgbfvzwkqj',
 'y': 'ealiortnscphumdgbfvkwxzqj',
 'z': 'ieaonrtlsdcmugphbyfvkwxqj'}

In [71]:
with open('count.txt','a') as f:
    f.write("----------------------------------\n")
    for i in sorted(c1x1.keys()):
        f.write(c1x1[i])
        f.write('\n')        

In [87]:
c2x1=count_2x1(wordlist)

In [92]:
c2x1

{'aa': 'rsinltemchbduokgpvfwyzjxq',
 'as': 'eitrloncpumhdgbyfqvkwzxj',
 'ac': 'eitrosnlhumdpbykgfvwjzxq',
 'ch': 'eaiorsntlmupdygbkfwvzxqj',
 'he': 'tarsioncldpmuygbwfkvxzqj',
 'en': 'tsiarolcdmupghybvfwkzxqj',
 'ae': 'iorcltndsphmugybvfzxkjqw',
 'ee': 'rsntlidaohgpcwubmfkyvxzqj',
 'ag': 'einrsotlmpudchybvfwkzxjq',
 'ah': 'erinsotldmucbpkygwzjvfxq',
 'ed': 'rinatlsoucphmbgfwvykzxqj',
 'hs': 'ateoirnclugpmdbkwfyvzjqx',
 'al': 'ieontsrcpmuhydgbvfzkwxqj',
 'le': 'sainrtocdbupmghyvfwkxzqj',
 'es': 'intraolcudphmgbfvywkzxqj',
 'su': 'erinatlpobcdmgyfhvkzwxjq',
 'un': 'eiarstldocgpmbyhfvkwzjxq',
 'nd': 'eiarosultcmgphbyfwvkzxqj',
 'li': 'eantsorcupgmdbhyfkvzwqxj',
 'ii': 'aendtrslocumhpgybqfvxwkzj',
 'is': 'etaonrlmchdpugybvfwkxqzj',
 'ls': 'eaiorntuphcdmfbwgvykxqzj',
 'st': 'eiaornlcupdmhgybfwvkzxjq',
 'am': 'eirnosltcdphugbyfkwzvxjq',
 'an': 'ietrsolcmdhpugybfkvwzxjq',
 'da': 'einolrtscmupyhgbfvkwzxjq',
 'hl': 'eoisacrtnypdmugbkfwvzxqj',
 'ao': 'irtnslecughpmdbykfxvwzjq',
 'ap': 'eroisthnl

In [89]:
for i in c2x1.keys():
    l=c2x1[i]
    ls=sorted(l.items(), key=lambda x: x[1], reverse=True)
    s=""
    for j in ls:
        s+=j[0]
    c2x1[i]=s

In [93]:
with open('count.txt','a') as f:
    f.write("----------------------------------\n")
    for i in sorted(c2x1.keys()):
        f.write(i+" : "+c2x1[i])
        f.write('\n')

In [98]:
c3x1=count_3x1(wordlist)

In [103]:
c3x1

{'aaa': 'wskf',
 'aas': 'eivtnbkrhmcgpolf',
 'aac': 'nistorhegupw',
 'ach': 'eriotsnlmpybdugvfkwzxqj',
 'che': 'rastionldumpbykgwfvzqxj',
 'hen': 'iaopstrcldmyugbzkwvfxqj',
 'aae': 'rbdsupil',
 'aee': 'lopinthscgdybm',
 'aag': 'erlminvhdsputw',
 'aah': 'esdcn',
 'ahe': 'rdtoinclspxmkyugzbfvwj',
 'hed': 'astrocnilupmbwgfyvkxqzj',
 'ahs': 'liorpmetudvbcnkzygfjw',
 'aal': 'rseidbtnkmugjhzcvwxyopq',
 'ale': 'sniotrcdphmugvybkwfxqzj',
 'les': 'ationrcpudhbmgfywkvzxjq',
 'esu': 'ritpnloacdmbgyfvhjzwkx',
 'sun': 'ediartolgbhmyckwpfvzxqj',
 'und': 'eroisatlgcbpmhfywvkzxjq',
 'ali': 'entsrocmudpzyghvbfkqxwj',
 'lii': 'aedonpsrtmcfbguyhzv',
 'iis': 'atonmrbdeluchgpzqvkfw',
 'als': 'eirtnocpmudfhbgvywkqxz',
 'lst': 'eaorinduhpwmfcgbkyvj',
 'aam': 'lstnoriceuphbgkyxdj',
 'aan': 'ictesrhlbpgmjokudfyvx',
 'and': 'ersliothmgcubpywfkvzxjq',
 'nda': 'esirloutmcbgpyhfzvkwxqj',
 'dah': 'elnisourbgytkmwzvcjpq',
 'ahl': 'tseidnmroukgbpfzwvcyj',
 'aao': '',
 'aap': 'snebcirktdul',
 'aps': 'leicrntoudhygkmbw

In [100]:
for i in c3x1.keys():
    l=c3x1[i]
    ls=sorted(l.items(), key=lambda x: x[1], reverse=True)
    s=""
    for j in ls:
        s+=j[0]
    c3x1[i]=s

In [104]:
with open('count.txt','a') as f:
    f.write("----------------------------------\n")
    for i in sorted(c3x1.keys()):
        f.write(i+" : "+c3x1[i])
        f.write('\n')